nltk
sklearn

house / Resident 

# POS inside the Lemmatization & manual 
# english dictonary 

In [3]:
import pandas as pd 
data=pd.read_csv(r"D:\Python Code\Text Mining Msc\Text_Similarity.csv",encoding='latin-1')

data

,text
0,We went to the pizza place and you ate no pi...
1,I ate pizza with you yesterday at home
2,There no place like home


# Lower Case 

In [4]:
data.text=data.text.str.lower()
data

,text
0,we went to the pizza place and you ate no pi...
1,i ate pizza with you yesterday at home
2,there no place like home


In [5]:
# Import stopwords with nltk.
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() 
                                                      if word not in (stop)]))
data

,text
0,went pizza place ate pizza
1,ate pizza yesterday home
2,place like home


In [6]:
import nltk
from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

In [7]:
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w,"v") for w in w_tokenizer.tokenize(text)]

In [8]:
data.head()

,text
0,went pizza place ate pizza
1,ate pizza yesterday home
2,place like home


In [9]:
data['text'] = data.text.apply(lambda x :' '.join(lemmatize_text(x)))
data.head()

,text
0,go pizza place eat pizza
1,eat pizza yesterday home
2,place like home


# DTM 

In [12]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())

# One of the required inputs is a term-document matrix
tdm = data_dtm
tdm

,eat,home,like,pizza,place,yesterday
0,1,0,0,2,1,0
1,1,1,0,1,0,1
2,0,1,1,0,1,0


In [50]:
from sklearn.metrics.pairwise import cosine_similarity  # Compute cosine similarity between samples in X and Y.

find_similarities = cosine_similarity(tdm) # We are transposing the matrix 
print (find_similarities)

[[1.         0.61237244 0.23570226]
 [0.61237244 1.         0.28867513]
 [0.23570226 0.28867513 1.        ]]


In [51]:
pd.DataFrame(find_similarities,columns=["Doc1","Doc2","Doc3"],index=["Doc1","Doc2","Doc3"])

,Doc1,Doc2,Doc3
Doc1,1.000000,0.612372,0.235702
Doc2,0.612372,1.000000,0.288675
Doc3,0.235702,0.288675,1.000000


* Doc1 & Doc2 IS RELATED TO EACH OTHER 

# TF-IDF Method

In [13]:
from sklearn.feature_extraction.text import CountVectorizer 
cv=CountVectorizer(stop_words='english') 
word_count_vector=cv.fit_transform(data.text)

feature_names = cv.get_feature_names_out()  ## EVERY WORD NAME
feature_names

array(['eat', 'home', 'like', 'pizza', 'place', 'yesterday'], dtype=object)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer 
 
tfidf_vectorizer = TfidfVectorizer(use_idf=True,stop_words='english') 

# just send in all your docs here 
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(data.text)

row_name=["DOC1","DOC2","DOC3"]

output=pd.DataFrame.sparse.from_spmatrix(tfidf_vectorizer_vectors,
                                         index=row_name,
                                         columns=feature_names)
output

,eat,home,like,pizza,place,yesterday
DOC1,0.408248,0.000000,0.000000,0.816497,0.408248,0.000000
DOC2,0.459854,0.459854,0.000000,0.459854,0.000000,0.604652
DOC3,0.000000,0.517856,0.680919,0.000000,0.517856,0.000000


Output2=tfidf_vectorizer_vectors.todense()
Output2=pd.DataFrame(Output2,columns=list(feature_names),index=row_name)
Output2.head()

In [55]:
find_similarities = cosine_similarity(output) # We are transposing the matrix 
pd.DataFrame(find_similarities,columns=["Doc1","Doc2","Doc3"],index=["Doc1","Doc2","Doc3"])

,Doc1,Doc2,Doc3
Doc1,1.000000,0.563203,0.211414
Doc2,0.563203,1.000000,0.238138
Doc3,0.211414,0.238138,1.000000


* Doc1 & Doc2 with 56% IS RELATED TO EACH OTHER 